In [1]:
import numpy as np
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep

In [4]:
requests.get('http://www.politickefinance.cz/robots.txt')

<Response [200]>

In [7]:
print(requests.get('http://www.politickefinance.cz/robots.txt').text)

In [108]:
# Gradual testing - not final version, chci spat
Instance_Finance = External('http://www.politickefinance.cz')
Instance_Finance.getSoup() # OK
Instance_Finance.getName() # OK
Instance_Finance.getAllLinks() # OK

['http://www.politickefinance.cz/cps/zebricek-stran/',
 'http://www.politickefinance.cz/sz/zebricek-stran/',
 'http://www.politickefinance.cz/sso/zebricek-stran/',
 'http://www.politickefinance.cz/top09/zebricek-stran/',
 'http://www.politickefinance.cz/kscm/zebricek-stran/',
 'http://www.politickefinance.cz/stan/zebricek-stran/',
 'http://www.politickefinance.cz/kducsl/zebricek-stran/',
 'http://www.politickefinance.cz/cssd/zebricek-stran/',
 'http://www.politickefinance.cz/ods/zebricek-stran/',
 'http://www.politickefinance.cz/spo/zebricek-stran/',
 'http://www.politickefinance.cz/spd/zebricek-stran/',
 'http://www.politickefinance.cz/ano11/zebricek-stran/']

In [107]:
class External:
    '''
    Downloader manager for External lecturers from IES
    
    It contains methods for collection of links, downloading itself and storing results
    '''
    def __init__(self, link):
        '''
        Creates the External object with self.links that store links to be downloaded
        to store individual people teaching at IES
        '''
        self.link = link
        
    def getSoup(self):
         
        r = requests.get(self.link)
        r.encoding = "UTF-8"
        soup = BeautifulSoup(r.text, "lxml")
        return soup
    
    def getName(self):
        '''Extract the name of the political party.'''
        soup2 = self.getSoup().findAll("div", {"class": "table-item table-item--name is-uppercase"})
        names = [s.text.strip() for s in soup2]
        return names

    def getAllLinks(self):
        r = requests.get(self.link)
        r.encoding = "UTF-8"
        soup3 = BeautifulSoup(r.text, "lxml").findAll("a", {"class": "flex-row"})
        links = []
        for li in soup3:
            lili = ("http://www.politickefinance.cz" + li["href"])
            links.append(lili)
        return links
    

In [80]:
# Gradually solely not by OOP - Soup:
def getSoup(link):
    sleep(0.1) #to be kind to the website
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml')

Bf = getSoup('http://www.politickefinance.cz/cps/zebricek-stran/')

In [84]:
Tds = Bf.find('div', {'class':'mark'})
Tds.text.strip()

'1'

In [15]:
Tds[0]

<div class="table-item table-item--name is-uppercase">              Piráti</div>

In [10]:
tds = Bf.find('div', {'class':'table-item table-item--name is-uppercase'})
tds

<div class="table-item table-item--name is-uppercase">              Piráti</div>

In [9]:
idx = np.linspace(1, 78, 78)
idx = np.round(idx)
idx = idx.astype(int)
idx

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78])

In [ ]:
  def getID(self):
        '''Extract the ID of lecturer.'''
           # 'https://ies.fsv.cuni.cz/en/staff/zeleny'
        IDs = [id.split('/')[-1] for id in self.getAllLinks()]
        return IDs
    
    def getNextSiblingOfStrong(self, characteristic):
        '''
        Function for the extraction of characteristics for lecturers
        These are then specified in saveDFs function
        '''
        si = []
        for li in self.getAllLinks():
            si1 = requests.get(li)
            si1.encoding = "UTF-8"
            si2 = BeautifulSoup(si1.text, "lxml").find("strong", text = characteristic).next_sibling.strip()
            si.append(si2)
            
            return si
        
    def saveDFs(self):
        '''Specifying characteristics for lecturers and generate Panda Data Frame.'''
        names = self.getName()
        ids = self.getID()
        pos = self.getNextSiblingOfStrong("Position:")
        phones = self.getNextSiblingOfStrong("Phone:")
        office = self.getNextSiblingOfStrong("Office:")
        mail = self.getNextSiblingOfStrong("Email:")
        df = pd.DataFrame({
            "Name": names, 
            "Position": pos,
            "Phone": phones,
            "E-mail": mail}, index = ids)
        
        return df